
# Chat



TODO: move chat analysis tools here, separate from brian's resume, generalize so history can be found in discord and both bots can benefit from the additional context.




## Persistance




### store conversation

store llm response?

store chat history so that it can be recalled later and added to the context window, this gives the LLM more context about how the conversation is going.


In [ ]:
const path = require('path')
const fs = require('fs')
const selectModel = importer.import('select llm')
const {askLlamaAboutEmotions} = importer.import('ask llm about emotions')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH, DEFAULT_MODEL} = importer.import('general chit chat')


async function storeResponse(user, session, content, context, otr) {
  let promptModel = await selectModel(DEFAULT_MODEL)

  if(!session) {
    return {
      emotions: await askLlamaAboutEmotions(content)
    }
  }

  let now = new Date()
  let convoFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') 
    + '-' + DEFAULT_MODEL
    + '-' + session + '.json')
  if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
    if(fs.existsSync(convoFile)) {
      ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
    } else {
      ACTIVE_CONVERSATIONS[convoFile] = {}
    }
  }

  let contextContainsImage = false
  if(context && context.startsWith('data:image/png;base64,')) {
    contextContainsImage = true
  }

  let summary
  if(!otr) {
    summary = await promptModel('Summerize this prompt in one short sentence:\n' 
      + content + '\nOnly respond with the summary, no pleasantries.')
  }
  let keywords = await promptModel('List a few key words that categorize this prompt:\n' 
    + content + '\nOnly respond with a single category, no pleasantries.')
  let emotions = await askLlamaAboutEmotions(content)

  let result = ACTIVE_CONVERSATIONS[convoFile][Date.now()] = {
    user: user,
    content: otr ? void 0 : content,
    context: contextContainsImage ? void 0 : context,
    summary: summary,
    keywords: keywords,
    emotions: emotions,
    otr: otr ? true : false,
  }
  fs.writeFileSync(convoFile, JSON.stringify(ACTIVE_CONVERSATIONS[convoFile], null, 4))

  return result
}


module.exports = storeResponse



### general discussion

general chit chat?

everything else falls into this category. context will be populated with recent message conversation history.


In [ ]:
const path = require('path')
const fs = require('fs')


const PROJECT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'conversations')
const ACTIVE_CONVERSATIONS = {}
const DEFAULT_MODEL = process.env.DEFAULT_MODEL || 'Default'

// TODO: load some previous summaries and contents

async function messageRecents(promptModel, session) {

  if(!session) {
    return ''
  }

  let now = new Date()
  let convoFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') 
    + '-' + DEFAULT_MODEL
    + '-' + session + '.json')
  // TODO: reload chat
  if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
    if(fs.existsSync(convoFile)) {
      ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
    } else {
      ACTIVE_CONVERSATIONS[convoFile] = {}
    }
  }

  let messageTimes = Object.keys(ACTIVE_CONVERSATIONS[convoFile])
    .filter(k => k != 'summaries' && k != 'memories')
  messageTimes.sort((a, b) => b - a)
  let count = 0
  let messages = 'Current date: ' + (new Date).toISOString() 
    + '\nOur recent conversion:\n'
  for(let i = 0; i < messageTimes.length && count < 25; i++) {
    let message = ACTIVE_CONVERSATIONS[convoFile][messageTimes[i]]
    let useSummary = i > 10 || messages.length > 1024 || (message.content && message.content.length > 1024)
    let useKeywords = !message.content
    messages += (message.user ? 'User: ' : 'AI: ') 
      + (useKeywords ? message.keywords : (useSummary ? message.summary : message.content))
      + '\n\n'
    if(messages.length > 2048) {
      break
    }
    count++
  }

  return messages
}

module.exports = {
  messageRecents,
  PROJECT_PATH,
  ACTIVE_CONVERSATIONS,
  DEFAULT_MODEL
}



### load history

relevant llm history?

Ask the llm to pick out history files that are related to the current prompt. This way if they refer to last month, week, yesterday, the correct history files will be loaded.


In [ ]:
const path = require('path')
const fs = require('fs')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH, DEFAULT_MODEL} = importer.import('general chit chat')


async function relevantHistory(promptModel, session, prompt) {

  if(!session) {
    return ''
  }

  // TODO: extract time frame to, from prompt
  let history = fs.readdirSync(PROJECT_PATH)
  let historyFiles = []
  for(let i = 0; i < history.length; i++) {
    if(history[i].match('-' + DEFAULT_MODEL + '-' + session + '.json')) {
      historyFiles.push(path.basename(history[i]))
    }
  }

  let now = new Date()
  let q1 = 'Current date: ' + now.getFullYear() + '-' + String(now.getMonth() + 1).padStart(2, '0')
    + '\nGiven the following chat history:\n'
    + historyFiles.map(f => f.substring(0, 7)).join('\n')
    + '\nGiven the following prompt:\n'
    + prompt + '\nReturn a short range of Year-Month for related dates.'
  console.log('User: ' + q1)
  let a1 = await promptModel(q1)
  console.log('AI: ' + a1)

  // TODO: create a range of month out of the response
  let relevantFiles = []
  for(let i = 0; i < historyFiles.length; i++) {
    if(a1.match(historyFiles[i])) {
      let convoFile = path.join(PROJECT_PATH, historyFiles[i])
      if(fs.existsSync(convoFile)) {
        ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
      } else {
        ACTIVE_CONVERSATIONS[convoFile] = {}
      }
      relevantFiles.push(historyFiles[i])
    }
  }

  return relevantFiles
}

module.exports = relevantHistory




### chat keywords

relevant chat keywords?

Pick out the relevant keywords to search in chat history.


In [ ]:
const path = require('path')
const fs = require('fs')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH, DEFAULT_MODEL} = importer.import('general chit chat')


async function relevantKeywords(promptModel, session, prompt) {
  
  // TODO: search all loaded user histories for keywords
  let keywords = []
  let loadedConversations = Object.keys(ACTIVE_CONVERSATIONS)
    .filter(key => key.match('-' + DEFAULT_MODEL + '-' + session + '.json'))
  loadedConversations.sort((a, b) => b - a)
  for(let i = 0; i < loadedConversations.length; i++) {
    let conversation = ACTIVE_CONVERSATIONS[loadedConversations[i]]
    let timestamps = Object.keys(conversation).filter(k => k != 'summaries' && k != 'memories')
    timestamps.sort((a, b) => b - a)
    for(let j = 0; j < timestamps.length; j++) {
      let message = conversation[timestamps[j]]
      let newKeywords = message.keywords
      newKeywords = newKeywords.replace(/.*?:\s*\n\s*/gi, '')
      newKeywords = newKeywords.replaceAll('*', '').replaceAll('# ', '').replaceAll('#', '').replaceAll(/[0-9]+\.\s*/gi, '')
      newKeywords = newKeywords.split(/\s*\n\s*|\s*,\s*/gi)
      keywords = keywords.concat(newKeywords.map(k => k.trim()))
    }
  }
  keywords = keywords.filter((k, i, arr) => k && arr.indexOf(k) == i)


  let q2 = 'Given the following keywords:\n'
    + keywords.join('\n')
    + '\nGiven the following prompt:\n'
    + prompt + '\nReturn only a few keywords related to the prompt.'
  console.log('User: ' + q2)
  let a2 = await promptModel(q2)
  console.log('AI: ' + a2)

  let matching = []
  for(let i = 0; i < keywords.length; i++) {
    if(a2.match(keywords[i])) {
      matching.push(keywords[i])
    }
  }

  return matching
}

module.exports = relevantKeywords


### specific timestamps

relevant history timestamps?

Get a list of timestamps that might be related to the current prompt.


In [ ]:
const {ACTIVE_CONVERSATIONS, PROJECT_PATH, DEFAULT_MODEL} = importer.import('general chit chat')

async function askLlamaMatchTimestamps(promptModel, messages, keywords, prompt, timestamps) {
  let q3 = 'Given the following keywords:\n'
    + keywords.join('\n')
    + '\n' + messages
    + prompt + '\nOnly respond with related and unique timestamps, no explanations.'
  console.log('User: ' + q3)
  let a3 = await promptModel(q3)
  console.log('AI: ' + a3)

  return timestamps
    .filter(time => a3.match(time) || a3.match(new Date(parseInt(time)).toISOString()))
}


async function matchingTimestamps(promptModel, session, prompt, keywords) {
  
  let matchingTimestamps = []
  let messages = 'Current date: ' + (new Date).toISOString() 
    + '\nOur recent topics:\n'
  let originalTimestamp = messages
  let loadedConversations = Object.keys(ACTIVE_CONVERSATIONS)
    .filter(key => key.match('-' + DEFAULT_MODEL + '-' + session + '.json'))
  for(let i = 0; i < loadedConversations.length; i++) {
    let conversation = ACTIVE_CONVERSATIONS[loadedConversations[i]]
    let timestamps = Object.keys(conversation).filter(k => k != 'summaries' && k != 'memories')
    timestamps.sort((a, b) => b - a)
    for(let j = 0; j < timestamps.length; j++) {
      let message = conversation[timestamps[j]]
      let topics = keywords.filter(key => message.keywords.match(key))
      if(!prompt.match(timestamps[j]) && topics.length == 0) {
        continue
      }

      messages += new Date(parseInt(timestamps[j])).toISOString() 
        + ' - ' + topics.join(', ') 
        + (message.summary ? (' - ' + message.summary) : '') 
        + '\n'

      if(messages.length > 2048) {
        let newTimestamps = await askLlamaMatchTimestamps(promptModel, messages, keywords, prompt, timestamps)

        matchingTimestamps = matchingTimestamps.concat(newTimestamps)

        messages = originalTimestamp
      }
    }

    if(messages.length > originalTimestamp.length) {
      let newTimestamps = await askLlamaMatchTimestamps(promptModel, messages, keywords, prompt, timestamps)

      matchingTimestamps = matchingTimestamps.concat(newTimestamps)
    }
  }

  return matchingTimestamps
}

module.exports = matchingTimestamps




## Conversation Tools



### classify prompt

classify llm prompt?

A basic entry point to allow LLM to access select notebook functions.

TODO: merge this code with "ask llm matching function"


In [ ]:
const getExports = importer.import('get exports from source')


const API_FUNCTIONS = [
  'load message history',
  'general chit chat',
  'brians resume',
  'file system access',
  'access web information',
  'stable diffusion request',
  'request ollama vision',
  //'ask another llm for help',
  'add conversation context',
  'llm save memories'
  // 'switch llm models'
]

const API_EXPORTS = API_FUNCTIONS.map(func => getExports(importer.interpret(func).code)[0])

const API_DESCRIPTION = API_FUNCTIONS
  .map(func => importer.interpret(func))
  .filter((code, i) => API_EXPORTS[i])
  .map((code, i) => API_EXPORTS[i] + ' - ' + code.markdown.join('').trim()
    .replaceAll(/^#.*?\n/gi, '')
    .replaceAll(code.questions, '')
    .replaceAll(/\s*[\n]+[\s\r\n]*/gi, '-'))
  .join('\n')

async function askLlamaMatchingFunction(promptModel, prompt, image) {

  let q1 = 'Given the following functions:\n'
    + API_DESCRIPTION + '\nDo any of these functions relate to this prompt:\n'
    + (image ? 'An image has been provided as a part of the input.' : '')
    + prompt + '\nOnly return the function names in order of relevance.'
  console.log('User: ' + q1)
  let a1 = await promptModel(q1)
  console.log('AI: ' + a1)

  let matchingFunctions = a1.split(/\s*\n\s*|\s*,\s*/gi)
    .map((matchStr) => {
      for (let i = 0; i < API_FUNCTIONS.length; i++) {
        if (!API_EXPORTS[i]) {
          continue
        }
        if (API_EXPORTS[i].includes(matchStr) || matchStr.match(API_EXPORTS[i])) {
          return ({
            importStr: API_FUNCTIONS[i],
            exportStr: API_EXPORTS[i]
          })
        }
      }
    })
    .filter(func => a1.length > 0 && func)
    .map(func => func.importStr)

  return matchingFunctions
}

module.exports = {
  askLlamaMatchingFunction,
  API_FUNCTIONS,
  API_EXPORTS,
  API_DESCRIPTION,
}




### conversation handler

handle conversation?



In [ ]:
const classifyPrompt = importer.import('add conversation context')
const storeResponse = importer.import('store llm response')

async function handleConversation(promptModel, session, prompt, image, otr) {
  let context = await classifyPrompt(promptModel, session, prompt, image)

  let messageStorage = await storeResponse(true, session, prompt, 
    context.content + (context.memories ? ('\n' + context.memories.join('\n')) : ''), 
    otr)

  let q1 = (context && context.content ? context.content : '') 
  + (context.image ? '\nThe response image is provided.\n' : '')
  + '\nRespond to this message and pretend to be emotional (e.g. ' 
  + messageStorage.emotions.join(', ') + '):\n' 
  + prompt
  + (context.memories ? ('\nApply any relevant memories to your reponse:\n' + context.memories.join('\n')) : '')
  console.log('User: ' + q1)
  let result = await promptModel(q1)

  if(context.imagePath) {
    if(result.match(/!\[[^\]]*\]\((.*?)\s*("(?:.*[^"])")?\s*\)/gi)) {
      result = result.replace(/!\[[^\]]*\]\((.*?)\s*("(?:.*[^"])")?\s*\)/gi, 
        // TODO: accomodate discord by removing the markdown? 
        //   or providing a server address to brian-chat? 
        //   return entire context with result to doMention()?
        '![' + context.prompt + '](/' + context.imagePath.replaceAll(' ', '%20') + ')')
    } else {
      result += '\n\n![' + context.prompt + '](/' + context.imagePath.replaceAll(' ', '%20') + ')'
    }
  }

  if(session) {
    await storeResponse(false, session, result, void 0, otr)
  }

  return result
}

module.exports = handleConversation




### add conversation

add conversation context?

create a conversation context by selecting a runnable function available to call by the llm. this is the entry point for all inquiries about functionality or what functions are available.


In [ ]:
const { messageRecents } = importer.import('general chit chat')
const {doStableRequest} = importer.import('stable diffusion request')
//const messageOllamaVision = importer.import('describe an image')
const {askLlamaMatchingFunction, API_DESCRIPTION} = importer.import('classify llm prompt')
const getParameters = importer.import('function parameters')
const {listMemories} = importer.import('llm load memories')

async function classifyPrompt(promptModel, session, prompt, image, otr) {

  let matchingFunctions = await askLlamaMatchingFunction(promptModel, prompt, image)

  let importedFunction
  let answer = ''
  let context = {
    promptModel, session, prompt, image, otr
  }

  // drop out early if the matching function is ourselves, this is how we return our function description for inquiries about capabilities.
  if(matchingFunctions == classifyPrompt) {
    return {
      memories: await listMemories(session),
      content: 'Given the following functions:\n' + API_DESCRIPTION
    }
  }

  //let historyFiles = await relevantHistory(promptModel, session, prompt)

  // TODO: convert to available parameters like Core/import.ipynb:run() style parameterization
  for (let i = 0; i < matchingFunctions.length; i++) {
    importedFunction = importer.import(matchingFunctions[i])
    if (typeof importedFunction == 'object' && typeof Object.values(importedFunction)[0] == 'function') {
      importedFunction = Object.values(importedFunction)[0]
    }
    // call parameterized
    let params = getParameters(importedFunction.toString()).slice(1)
    let inputs = []
    for(let j = 0; j < params.length; j++)
      inputs[j] = context[params[j]]
    answer = await importedFunction.apply(null, inputs)
    break
  }

  if(importedFunction == doStableRequest) {
    return Object.assign(answer, {
      memories: await listMemories(session),
      content: await messageRecents(session, prompt)
    })
  } else if (importedFunction != messageRecents) {
    if(typeof answer == 'object') {
      return Object.assign({}, {
        memories: await listMemories(session),
        content: await messageRecents(session, prompt)
      }, answer)
    } else {
      return {
        memories: await listMemories(session),
        content: (answer ? answer : '') + await messageRecents(session, prompt)
      }
    }
  } else if (!answer) {
    return {
      memories: await listMemories(session),
      content: await messageRecents(session, prompt)
    }
  } else {
    return {
      memories: await listMemories(session),
      content: answer
    }
  }

}

module.exports = classifyPrompt




## Memory bank

TODO: short and long term memory banks, short term can be a single month, not copied and for a single run/prompt stored by intermittent functions. long term are set by the user and copied from month to month.

Memories are appended after the prompt as additional instructions, these functions should ask for which memories are relevant intructions for the prompt. memories are applied to the last prompt in the sequence before returning to the user.



### loading memories

llm load memories?


In [ ]:
const fs = require('fs')
const path = require('path')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH, DEFAULT_MODEL} = importer.import('general chit chat')


async function listMemories(session) {

  let memories = await findMemories(session)
  return Object.keys(memories).map(key => {
    // provide the date the memory was made
    let date = new Date(parseInt(key))
    return date.getFullYear() + '-' + String(date.getMonth() + 1).padStart(2, '0') + '-' + date.getDate()
      + ' - ' + memories[key]
  })
}

async function findMemories(session) {

  if(!session) {
    return {}
  }

  let now = new Date()
  let currentFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') 
    + '-' + DEFAULT_MODEL
    + '-' + session + '.json')
  // TODO: reload chat
  if(typeof ACTIVE_CONVERSATIONS[currentFile] == 'undefined') {
    if(fs.existsSync(currentFile)) {
      ACTIVE_CONVERSATIONS[currentFile] = JSON.parse(fs.readFileSync(currentFile))
    } else {
      ACTIVE_CONVERSATIONS[currentFile] = {}
    }
  }

  if(typeof ACTIVE_CONVERSATIONS[currentFile]['memories'] != 'undefined') {
    debugger
    return ACTIVE_CONVERSATIONS[currentFile]['memories']
  }


  let history = fs.readdirSync(PROJECT_PATH)
  // newest to oldest
  history.sort((a, b) => b - a)
  for(let i = 0; i < history.length; i++) {
    if(!history[i].match('-' + DEFAULT_MODEL + '-' + session + '.json')) {
      continue
    }

    let convoFile = path.join(PROJECT_PATH, history[i])
    if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
      if(fs.existsSync(convoFile)) {
        ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
      } else {
        ACTIVE_CONVERSATIONS[convoFile] = {}
      }
    }

    let conversation = ACTIVE_CONVERSATIONS[convoFile]
    if(typeof conversation['memories'] != 'undefined') {
      return (ACTIVE_CONVERSATIONS[currentFile]['memories'] = conversation['memories'])
    }
  }

  return (ACTIVE_CONVERSATIONS[currentFile]['memories'] = {})
}

module.exports = {
  findMemories,
  listMemories,
}




### saving memories

llm save memories?

make changes to short and long term memories per the user's instructions. save a personal memory, always remember, remove a memory, modify internal memory, list recent memories.


In [ ]:
const fs = require('fs')
const path = require('path')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH, DEFAULT_MODEL} = importer.import('general chit chat')
const {findMemories, listMemories} = importer.import('llm load memories')


async function manageMemories(promptModel, session, prompt) {

  if(!session) {
    return ''
  }

  let now = new Date()
  let currentFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') 
    + '-' + DEFAULT_MODEL
    + '-' + session + '.json')
  // TODO: reload chat
  if(typeof ACTIVE_CONVERSATIONS[currentFile] == 'undefined') {
    if(fs.existsSync(currentFile)) {
      ACTIVE_CONVERSATIONS[currentFile] = JSON.parse(fs.readFileSync(currentFile))
    } else {
      ACTIVE_CONVERSATIONS[currentFile] = {}
    }
  }

  const MEMORY_FUNCTIONS = [
    'listMemories() - display a list of stored memories at the users request.',
    'saveMemory() - saves a new memory given the instructions to always remember something.',
    'removeMemory() - removes a matching memory from the storage bank.',
    'clearMemory() - removes all memories but only if the user is absolutely certain.',
  ]

  let q1 = 'Given the following memory related functions\n'
    + MEMORY_FUNCTIONS.join('\n')
    + '\nWhich function best matches this prompt?\n'
    + prompt
    + '\nOnly return the function name and nothing else.'
  let a1 = await promptModel(q1)

  if(a1.match('listMemories')) {

  } else if(a1.match('saveMemory')) {
    let q2 = 'Summerize this memory in one very precise sentance in first person:\n' + prompt + '\nOnly return the summary, no explanation.'
    let a2 = await promptModel(q2)

    ACTIVE_CONVERSATIONS[currentFile]['memories'] = await findMemories(session)
    ACTIVE_CONVERSATIONS[currentFile]['memories'][Date.now()] = a2
    fs.writeFileSync(currentFile, JSON.stringify(ACTIVE_CONVERSATIONS[currentFile], null, 4))
  } else if (a1.match('removeMemory')) {
    // TODO: make a functional list of memories
  } else if (a1.match('clearMemory')) {
    // TODO: did the user indicate they are absolutely sure about this action? Yes or No
    
  }
  return 'This is the list of available memories:\n' + await listMemories(session)
}

module.exports = manageMemories

